## Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np

In [2]:
from pandas_profiling import ProfileReport

# Carregando DataSets

In [3]:
ocorrencia_df = pd.read_csv('../DataSets/polished/ocorrencia.csv')
ocorrencia_tipo_df = pd.read_csv('../DataSets/polished/ocorrencia_tipo.csv')
aeronave_df = pd.read_csv('../DataSets/polished/aeronave.csv')
fator_contribuinte_df = pd.read_csv('../DataSets/polished/fator_contribuinte.csv')
recomendacao_df = pd.read_csv('../DataSets/polished/recomendacao.csv')

---------------------------------

## Visualizo dois caminhos gerais para aplicação de Machine Learning nesse DataSet:
- Classificar algo relacionado à ocorrência em si
- Classificar ou estruturar algo relacionado às recomendações

----------------------------

# Construção do DataFrame principal

## 1. Merge
Para contruir o nosso DataFrame principal vamos fazer um *merge* de cada DF usando a estrutura mostrada no site:  
http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/modelo_dados.png

In [4]:
cenipa0 = ocorrencia_df.merge(ocorrencia_tipo_df,how='inner',on='codigo_ocorrencia1')
cenipa1 = cenipa0.merge(aeronave_df,how='inner',on='codigo_ocorrencia2')
cenipa2 = cenipa1.merge(fator_contribuinte_df,how='inner',on='codigo_ocorrencia3')
cenipa_df = cenipa2.merge(recomendacao_df,on='codigo_ocorrencia4',how='inner')

#### É importante notar que, ao dar merge, linhas que não possuem key para ser mergeadas são excluídas. Podemos ver a diminuição de ocorrencias a partir da filtragem:

In [5]:
print('Merge número 1: {} ocorrências diferentes'.format(cenipa0['codigo_ocorrencia'].nunique()))
print('Merge número 2: {} ocorrências diferentes'.format(cenipa1['codigo_ocorrencia'].nunique()))
print('Merge número 3: {} ocorrências diferentes'.format(cenipa2['codigo_ocorrencia'].nunique()))
print('Merge número 4: {} ocorrências diferentes'.format(cenipa_df['codigo_ocorrencia'].nunique()))

Merge número 1: 5167 ocorrências diferentes
Merge número 2: 5167 ocorrências diferentes
Merge número 3: 944 ocorrências diferentes
Merge número 4: 561 ocorrências diferentes


**Por conta disso, aplicarei o cleaning aos dois datasets (cenipa1 e cenipa_df)**

## 2. Dropando duplicatas e Colunas de Merge
Eventualmente, ocorrem casos de duplicata no DataSet, por diversas razões. Vamos previnir isso.

In [6]:
cenipa1 = cenipa1.drop_duplicates().reset_index(drop=True)
cenipa_df = cenipa_df.drop_duplicates().reset_index(drop=True)

Não precisamos mais das colunas de *codigo_ocorrencia*

In [7]:
cenipa_df = cenipa_df.drop(columns=['codigo_ocorrencia',
                                   'codigo_ocorrencia1',
                                   'codigo_ocorrencia2',
                                   'codigo_ocorrencia3',
                                   'codigo_ocorrencia4'])

cenipa1 = cenipa1.drop(columns=['codigo_ocorrencia',
                               'codigo_ocorrencia1',
                               'codigo_ocorrencia2',
                               'codigo_ocorrencia3',
                               'codigo_ocorrencia4'])

## 3. Head, Info e Profiling
Agora vamos visualizar o *head*, o *info* e fazer o profiling do DataSet completo

### cenipa1

In [8]:
cenipa1.head(10)

,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_publicado,total_recomendacoes,...,aeronave_assentos,aeronave_ano_fabricacao,aeronave_registro_categoria,aeronave_registro_segmento,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_tipo_operacao,aeronave_nivel_dano,aeronave_fatalidades_total
0,INCIDENTE,PORTO ALEGRE - RS,RS,SBPA,1.0,20.0,NaN,FINALIZADA,NÃO,0,...,6.0,2003.0,AVIÃO,PARTICULAR,FORA DE AERODROMO,FORA DE AERODROMO,POUSO,PRIVADA,LEVE,0
1,ACIDENTE,GUARULHOS - SP,SP,SBGR,1.0,13.0,SIM,FINALIZADA,SIM,3,...,50.0,2001.0,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,DESCIDA,REGULAR,NENHUM,0
2,ACIDENTE,VIAMÃO - RS,RS,NaN,1.0,13.0,NaN,FINALIZADA,SIM,0,...,1.0,1976.0,AVIÃO,ESPECIALIZADA,FORA DE AERODROMO,FORA DE AERODROMO,ESPECIALIZADA,AGRÍCOLA,SUBSTANCIAL,0
3,ACIDENTE,SÃO SEBASTIÃO - SP,SP,NaN,1.0,17.0,NaN,NaN,NÃO,0,...,2.0,2007.0,ULTRALEVE,EXPERIMENTAL,FORA DE AERODROMO,FORA DE AERODROMO,CRUZEIRO,EXPERIMENTAL,LEVE,0
4,ACIDENTE,SÃO SEPÉ - RS,RS,NaN,1.0,16.0,SIM,FINALIZADA,SIM,0,...,1.0,1986.0,AVIÃO,ESPECIALIZADA,FORA DE AERODROMO,FORA DE AERODROMO,POUSO,AGRÍCOLA,SUBSTANCIAL,0
5,INCIDENTE,UBATUBA - SP,SP,NaN,1.0,14.0,NaN,FINALIZADA,NÃO,0,...,5.0,2000.0,HELICÓPTERO,PARTICULAR,FORA DE AERODROMO,FORA DE AERODROMO,CRUZEIRO,PRIVADA,LEVE,0
6,INCIDENTE GRAVE,CAMPINAS - SP,SP,SDAI,1.0,18.0,SIM,FINALIZADA,SIM,0,...,2.0,1940.0,AVIÃO,INSTRUÇÃO,FORA DE AERODROMO,FORA DE AERODROMO,DECOLAGEM,INSTRUÇÃO,SUBSTANCIAL,0
7,INCIDENTE,BELÉM - PA,PA,SBBE,1.0,19.0,NaN,FINALIZADA,NÃO,0,...,184.0,2008.0,AVIÃO,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,CORRIDA APÓS POUSO,REGULAR,LEVE,0
8,ACIDENTE,CONCEIÇÃO DAS ALAGOAS - MG,MG,NaN,1.0,16.0,NaN,FINALIZADA,NÃO,0,...,6.0,1982.0,AVIÃO,PARTICULAR,FORA DE AERODROMO,FORA DE AERODROMO,CRUZEIRO,PRIVADA,SUBSTANCIAL,0
9,INCIDENTE,UBERLÂNDIA - MG,MG,SBUL,1.0,22.0,NaN,FINALIZADA,NÃO,0,...,4.0,2011.0,AVIÃO,EXPERIMENTAL,FORA DE AERODROMO,FORA DE AERODROMO,POUSO,EXPERIMENTAL,LEVE,0


In [9]:
cenipa1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5417 entries, 0 to 5416
Data columns (total 37 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ocorrencia_classificacao        5417 non-null   object 
 1   ocorrencia_cidade               5417 non-null   object 
 2   ocorrencia_uf                   5415 non-null   object 
 3   ocorrencia_aerodromo            3431 non-null   object 
 4   ocorrencia_dia                  5416 non-null   float64
 5   ocorrencia_hora                 5416 non-null   float64
 6   investigacao_aeronave_liberada  3606 non-null   object 
 7   investigacao_status             5156 non-null   object 
 8   divulgacao_relatorio_publicado  5417 non-null   object 
 9   total_recomendacoes             5417 non-null   int64  
 10  total_aeronaves_envolvidas      5417 non-null   int64  
 11  ocorrencia_saida_pista          5417 non-null   object 
 12  ocorrencia_localizacao          38

In [10]:
prof_cenipa1 = ProfileReport(cenipa1)
prof_cenipa1.to_file(output_file='../Profiling/profiling_CENIPA1.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### cenipa_df

In [11]:
cenipa_df.head(10)

,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_publicado,total_recomendacoes,...,aeronave_nivel_dano,aeronave_fatalidades_total,fator_nome,fator_aspecto,fator_condicionante,fator_area,recomendacao_conteudo,recomendacao_status,recomendacao_destinatario_sigla,recomendacao_dias_para_feedback
0,ACIDENTE,GUARULHOS - SP,SP,SBGR,1.0,13.0,SIM,FINALIZADA,SIM,3,...,NENHUM,0,APLICAÇÃO DE COMANDOS,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL,Atuar junto às empresas que operam segundo o R...,CUMPRIDA,ANAC,117.0
1,ACIDENTE,GUARULHOS - SP,SP,SBGR,1.0,13.0,SIM,FINALIZADA,SIM,3,...,NENHUM,0,APLICAÇÃO DE COMANDOS,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL,Atuar junto à Administração do Aeroporto Inter...,CUMPRIDA,ANAC,117.0
2,ACIDENTE,GUARULHOS - SP,SP,SBGR,1.0,13.0,SIM,FINALIZADA,SIM,3,...,NENHUM,0,APLICAÇÃO DE COMANDOS,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL,Orientar as suas organizações subordinadas em ...,AGUARDANDO RESPOSTA,DECEA,NaN
3,ACIDENTE,GUARULHOS - SP,SP,SBGR,1.0,13.0,SIM,FINALIZADA,SIM,3,...,NENHUM,0,ATENÇÃO,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO,Atuar junto às empresas que operam segundo o R...,CUMPRIDA,ANAC,117.0
4,ACIDENTE,GUARULHOS - SP,SP,SBGR,1.0,13.0,SIM,FINALIZADA,SIM,3,...,NENHUM,0,ATENÇÃO,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO,Atuar junto à Administração do Aeroporto Inter...,CUMPRIDA,ANAC,117.0
5,ACIDENTE,GUARULHOS - SP,SP,SBGR,1.0,13.0,SIM,FINALIZADA,SIM,3,...,NENHUM,0,ATENÇÃO,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO,Orientar as suas organizações subordinadas em ...,AGUARDANDO RESPOSTA,DECEA,NaN
6,ACIDENTE,GUARULHOS - SP,SP,SBGR,1.0,13.0,SIM,FINALIZADA,SIM,3,...,NENHUM,0,CAPACITAÇÃO E TREINAMENTO,ASPECTO PSICOLÓGICO,ORGANIZACIONAL,FATOR HUMANO,Atuar junto às empresas que operam segundo o R...,CUMPRIDA,ANAC,117.0
7,ACIDENTE,GUARULHOS - SP,SP,SBGR,1.0,13.0,SIM,FINALIZADA,SIM,3,...,NENHUM,0,CAPACITAÇÃO E TREINAMENTO,ASPECTO PSICOLÓGICO,ORGANIZACIONAL,FATOR HUMANO,Atuar junto à Administração do Aeroporto Inter...,CUMPRIDA,ANAC,117.0
8,ACIDENTE,GUARULHOS - SP,SP,SBGR,1.0,13.0,SIM,FINALIZADA,SIM,3,...,NENHUM,0,CAPACITAÇÃO E TREINAMENTO,ASPECTO PSICOLÓGICO,ORGANIZACIONAL,FATOR HUMANO,Orientar as suas organizações subordinadas em ...,AGUARDANDO RESPOSTA,DECEA,NaN
9,ACIDENTE,GUARULHOS - SP,SP,SBGR,1.0,13.0,SIM,FINALIZADA,SIM,3,...,NENHUM,0,CLIMA ORGANIZACIONAL,ASPECTO PSICOLÓGICO,ORGANIZACIONAL,FATOR HUMANO,Atuar junto às empresas que operam segundo o R...,CUMPRIDA,ANAC,117.0


In [12]:
cenipa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6687 entries, 0 to 6686
Data columns (total 45 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ocorrencia_classificacao         6687 non-null   object 
 1   ocorrencia_cidade                6687 non-null   object 
 2   ocorrencia_uf                    6687 non-null   object 
 3   ocorrencia_aerodromo             3288 non-null   object 
 4   ocorrencia_dia                   6687 non-null   float64
 5   ocorrencia_hora                  6687 non-null   float64
 6   investigacao_aeronave_liberada   5168 non-null   object 
 7   investigacao_status              6687 non-null   object 
 8   divulgacao_relatorio_publicado   6687 non-null   object 
 9   total_recomendacoes              6687 non-null   int64  
 10  total_aeronaves_envolvidas       6687 non-null   int64  
 11  ocorrencia_saida_pista           6687 non-null   object 
 12  ocorrencia_localizac

In [13]:
prof_cenipa_df = ProfileReport(cenipa_df)
prof_cenipa_df.to_file(output_file='../Profiling/profiling_CENIPA_DF.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Salvando o DataFrame

Finalizamos a primeira etapa de *data wrangling*. Ainda iremos ajustar mais algumas coisas no DataSet antes de passar para uma eventual análise por *Machine Learning*. No momento, entretanto, já temos o suficiente para um entendimento geral do problema e para passar para uma etapa de Data Visualization de fato.

In [14]:
cenipa1.to_csv('../DataSets/merged/CENIPA1.csv',index_label=False)
cenipa_df.to_csv('../DataSets/merged/CENIPA_DF.csv',index_label=False)